In [2]:
import scanpy as sc
import pandas as pd
import numpy as np

In [2]:
# decompress raw data
!gzip -d "data/aggr/barcodes.tsv.gz" --force
!gzip -d "data/aggr/features.tsv.gz" --force
!gzip -d "data/aggr/matrix.mtx.gz" --force
!gzip -d "data/aggr/aggr_sample_barcodes/matrix.mtx.gz" --force

In [4]:
adata_aggr = sc.read("data/aggr/matrix.mtx")

In [5]:
# Load two more libraries with just entinostat and vehicle treated tumors
adata_wtile1 = sc.read("data/wtile1/matrix.mtx")
adata_wtile2 = sc.read("data/wtile2/matrix.mtx")
adata_wtilv1 = sc.read("data/wtilv1/matrix.mtx")
adata_wtilv2 = sc.read("data/wtilv2/matrix.mtx")
adata_aggr = adata_aggr.transpose()
adata_wtile1 = adata_wtile1.transpose()
adata_wtile2 = adata_wtile2.transpose()
adata_wtilv1 = adata_wtilv1.transpose()
adata_wtilv2 = adata_wtilv2.transpose()

In [9]:
# Add metadata
barcodes_aggr = pd.read_csv("data/aggr/barcodes.tsv", sep='\t', header=None, names=['barcode','run']) #need to have a run column for both
geneNames_aggr = pd.read_csv("data/aggr/features.tsv", sep='\t', header=None, names=['gene_id', 'gene_short_name', 'type'])
barcodes_wtile1 = pd.read_csv("data/wtile1/barcodes.tsv", sep='\t', header=None, names=['barcode','run']) #need to have a run column for both
barcodes_wtile2 = pd.read_csv("data/wtile2/barcodes.tsv", sep='\t', header=None, names=['barcode','run']) #need to have a run column for both
barcodes_wtilv1 = pd.read_csv("data/wtilv1/barcodes.tsv", sep='\t', header=None, names=['barcode','run']) #need to have a run column for both
barcodes_wtilv2 = pd.read_csv("data/wtilv2/barcodes.tsv", sep='\t', header=None, names=['barcode','run']) #need to have a run column for both
geneNames_pilot = pd.read_csv("data/wtilv1/features.tsv", sep='\t', header=None, names=['gene_id', 'gene_short_name', 'type'])

adata_aggr.obs_names = barcodes_aggr['barcode']
adata_aggr.var_names = geneNames_aggr['gene_id']
adata_aggr.var['gene_short_name'] = geneNames_aggr['gene_short_name'].values
adata_wtile1.obs_names = barcodes_wtile1['barcode']
adata_wtile1.var_names = geneNames_pilot['gene_id']
adata_wtile2.obs_names = barcodes_wtile2['barcode']
adata_wtile2.var_names = geneNames_pilot['gene_id']
adata_wtilv1.obs_names = barcodes_wtilv1['barcode']
adata_wtilv1.var_names = geneNames_pilot['gene_id']
adata_wtilv2.obs_names = barcodes_wtilv2['barcode']
adata_wtilv2.var_names = geneNames_pilot['gene_id']

In [6]:
adata_wtilv2

AnnData object with n_obs × n_vars = 4469 × 27998

In [9]:
adata_wtilv1

AnnData object with n_obs × n_vars = 1503 × 27998

In [10]:
adata_wtile1

AnnData object with n_obs × n_vars = 1129 × 27998

In [11]:
adata_wtile2

AnnData object with n_obs × n_vars = 964 × 27998

In [12]:
adata_aggr

AnnData object with n_obs × n_vars = 57029 × 27998
    var: 'gene_short_name'

In [10]:
adata = adata_aggr.concatenate(adata_wtile1, adata_wtile2, adata_wtilv1, adata_wtilv2, join='outer')
adata

AnnData object with n_obs × n_vars = 65094 × 27998
    obs: 'batch'
    var: 'gene_short_name-0'

In [13]:
#to join matrices for aggr and pilot, add anndata objects, on the union of observations
adata = adata_aggr.concatenate(adata_wtile1, adata_wtile2, adata_wtilv1, adata_wtilv2, join='outer')
adata
adata.write("aggr&pilot_preprocessed.h5ad", compression='gzip', compression_opts=1, force_dense=False)

/opt/conda/envs/Fertig_Python_3_7/lib/python3.7/site-packages/anndata/_io/h5ad.py:60: FutureWarning: The `force_dense` argument is deprecated. Use `as_dense` instead.
  FutureWarning,
... storing 'gene_short_name-0' as categorical


In [ ]:
# FILTERING STEPS
#adata = sc.read("aggr&pilot_preprocessed.h5ad") 

In [14]:
# Basic pre-processing
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)

In [15]:
mito_genes = adata.var['gene_short_name-0'].str.startswith('mt-')
# for each cell compute fraction of counts in mito genes vs. all genes
adata.obs['percent_mito'] = np.sum(adata[:, mito_genes].X, axis=1) / np.sum(adata.X, axis=1)
# add the total counts per cell as observations-annotation to adata
adata.obs['n_counts'] = adata.X.sum(axis=1)

In [16]:
#filtering
adata = adata[adata.obs['n_genes'] < 8000, :]
adata = adata[adata.obs['percent_mito'] < 0.15, :]

In [18]:
adata

View of AnnData object with n_obs × n_vars = 54636 × 19637
    obs: 'batch', 'n_genes', 'percent_mito', 'n_counts'
    var: 'gene_short_name-0', 'n_cells'

In [17]:
adata.var

,gene_short_name-0,n_cells
gene_id,,
ENSMUSG00000025902,Sox17,1034
ENSMUSG00000033845,Mrpl15,20731
ENSMUSG00000025903,Lypla1,14424
ENSMUSG00000104217,Gm37988,49
ENSMUSG00000033813,Tcea1,22778
ENSMUSG00000002459,Rgs20,3649
ENSMUSG00000085623,Gm16041,9
ENSMUSG00000033793,Atp6v1h,13020
ENSMUSG00000025907,Rb1cc1,15915


In [19]:
#save filtered barcodes
with open('filtered_barcodes.txt', 'w') as f:
    for item in adata.obs.index.values:
        f.write("%s\n" % item)
#save filtered genes
with open('filtered_genes_IDs.txt', 'w') as f:
    for item in adata.var.index.values:
        f.write("%s\n" % item)
        
with open('filtered_genes_short_name.txt', 'w') as f:
    for item in adata.var["gene_short_name-0"]:
        f.write("%s\n" % item)       

In [20]:
adata.write("aggr&pilot_filtered.h5ad", compression='gzip', compression_opts=1, force_dense=False)

/opt/conda/envs/Fertig_Python_3_7/lib/python3.7/site-packages/anndata/_io/h5ad.py:60: FutureWarning: The `force_dense` argument is deprecated. Use `as_dense` instead.
  FutureWarning,
